In [1]:
import pandas as pd
import numpy as np
import math
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
coords = pd.read_csv('/home/kamil/proga/digitdep_hack/geodata/buyer.csv')

In [3]:
coords['lat'] = pd.to_numeric(coords.geo.apply(lambda x: str(x).split(',')[0]), errors='coerce')
coords['lon'] = pd.to_numeric(coords.geo.apply(lambda x: str(x).split(',')[1] if len(str(x).split(',')) > 1 else np.nan), errors='coerce')
coords.dropna(inplace=True)

In [295]:
def dist(a, b):
    R = 6371.0

    a = torch.deg2rad(a)
    b = torch.deg2rad(b)

    dlat = (b - a)[:, 0]
    dlon = (b - a)[:, 1]

    if a.dim() == 1:
        a1 = torch.sin(dlat / 2)**2 + torch.cos(a[0]) * torch.cos(b[:, 0]) * torch.sin(dlon / 2)**2
    else:
        a1 = torch.sin(dlat / 2)**2 + torch.cos(a[:, 0]) * torch.cos(b[0]) * torch.sin(dlon / 2)**2
    c = 2 * torch.atan2(torch.sqrt(a1), torch.sqrt(1 - a1))

    distance = R * c
    
    return distance

In [5]:
def get_neighbours(data, dist_func, point, eps):
    dist = dist_func(data, point)
    return torch.nonzero(dist <= eps).flatten()

In [6]:
def expand(data, dist_func, i, c, eps, min_pts, labels, neighbours):
    labels[i] = c

    k = 0

    while k < len(neighbours):
        if labels[neighbours[k]] != -1:
            k += 1
            continue
        labels[neighbours[k]] = c
        n_k = get_neighbours(data, dist_func, data[neighbours[k]], eps)
        if len(n_k) >= min_pts:
            neighbours = torch.unique(torch.cat((neighbours, n_k)))

        k += 1 

    

In [7]:
def dbscan(data, dist_func, eps, min_pts):
    c = 0
    labels = torch.full((data.shape[0],), -1)
    for i in tqdm(range(data.shape[0])):
        if labels[i] != -1:
            continue # already visited
        neighbours = get_neighbours(data, dist_func, data[i], eps)
        if len(neighbours) < min_pts:
            labels[i] = -2 # label the point as noise
            continue
        
        c += 1
        expand(data, dist_func, i, c, eps, min_pts, labels, neighbours)
    return labels   
                

In [178]:
X = torch.tensor(coords[['lat', 'lon']].values, dtype=torch.int32)
labels = dbscan(X, dist, 1200, 3)

In [240]:
Q = np.vstack((np.array(X)[:, 0], np.array(X)[:, 1], np.array(labels))).swapaxes(0, 1)
lab = np.array(labels)
unique, counts = np.unique(lab, return_counts=True)
labels_dict = dict(zip(unique, counts))
labels_dict

{-2: 5, 1: 1550, 2: 40, 3: 7, 4: 12, 5: 21}

In [181]:
coords['label'] = labels

In [302]:
second_stage = coords[coords.label== max(labels_dict, key=labels_dict.get)]

In [303]:
#coords.loc[coords.label==1, 'label'] = second_stage['test']

In [333]:
second_x = torch.tensor(second_stage[['lat', 'lon']].values, dtype=torch.int32)
second_labels = dbscan(second_x, dist, 250, 30)

  0%|          | 0/1590 [00:00<?, ?it/s]/tmp/ipykernel_43129/650074921.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  a = torch.deg2rad(torch.tensor(a))
/tmp/ipykernel_43129/650074921.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.deg2rad(torch.tensor(b))
100%|██████████| 1590/1590 [00:00<00:00, 3912.97it/s]


In [334]:
second_Q = np.vstack((np.array(second_x)[:, 0], np.array(second_x)[:, 1], np.array(second_labels))).swapaxes(0, 1)
second_lab = np.array(second_labels)
second_unique, second_counts = np.unique(second_lab, return_counts=True)
dict(zip(second_unique, second_counts))

{-2: 184,
 1: 88,
 2: 431,
 3: 54,
 4: 18,
 5: 23,
 6: 78,
 7: 297,
 8: 203,
 9: 44,
 10: 1,
 11: 122,
 12: 1,
 13: 6,
 14: 1,
 15: 39}

In [ ]:
second_stage['label'] = second_labels

In [338]:
third_stage = second_stage[second_stage.label == min(second_stage.label)]

In [352]:
third_x = torch.tensor(third_stage[['lat', 'lon']].values, dtype=torch.int32)
third_labels = dbscan(third_x, dist, 700, 5)
third_Q = np.vstack((np.array(third_x)[:, 0], np.array(third_x)[:, 1], np.array(third_labels))).swapaxes(0, 1)
third_lab = np.array(third_labels)
third_unique, third_counts = np.unique(third_lab, return_counts=True)
dict(zip(third_unique, third_counts))

  0%|          | 0/162 [00:00<?, ?it/s]/tmp/ipykernel_43129/650074921.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  a = torch.deg2rad(torch.tensor(a))
/tmp/ipykernel_43129/650074921.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  b = torch.deg2rad(torch.tensor(b))
100%|██████████| 162/162 [00:00<00:00, 2862.35it/s]


{-2: 2, 1: 50, 2: 51, 3: 3, 4: 8, 5: 29, 6: 7, 7: 1, 8: 11}

In [353]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Пример данных с числовыми значениями

# Нормализация значений для генерации цветов
values = [value for _, _, value in third_Q]
norm = mcolors.Normalize(vmin=min(values), vmax=max(values))
cmap = cm.get_cmap('inferno')  # Выбор цветовой карты

# Создаем карту
m = folium.Map(location=[54.5, 75.5], zoom_start=3)

# Добавляем маркеры с цветами на основе значений
for lat, lon, value in third_Q:
    color = mcolors.to_hex(cmap(norm(value)))  # Преобразуем в HEX цвет
    folium.CircleMarker(
        location=[lat, lon],
        radius=8,
        color=color,
        fill=True,
        fill_opacity=0.7,
        tooltip=str(value)
    ).add_to(m)

# Сохраняем карту в HTML
m.save("automatically_colored_markers_map.html")
